This notebook relies on the following cmd line tools, ensure they are installed and in the system
- tippecanoe
- mapshaper
- aws cli

In [1]:
from pathlib import Path
import sys
import logging

from IPython.lib import backgroundjobs as bg

scripts_dir = Path('..').joinpath('src')
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

from pipelines.pipes import get_pipes, execution_order, filter_pipes

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("fiona").setLevel(logging.WARNING)
jobs = bg.BackgroundJobManager()

In [2]:
# def execute_pipes(pipes):
#     ordered_pipes = execution_order(pipes)
#     for pipe in ordered_pipes:
#         jobs.new(pipe().execute)

# # this code will execute all pipelines selected in background.

# mypipes_subset = filter_pipes(get_pipes().items(), ['eez_tiles'])
# execute_pipes(mypipes_subset)

In [3]:
get_pipes()

{'mpa_intermediate': pipelines.intermediate_pipes.MpasIntermediatePipe.EEZIntermediatePipe,
 'mpaatlas_intermediate': pipelines.intermediate_pipes.MpaAtlasIntermediatePipe.MpaAtlasIntermediatePipe,
 'eez_tiles': pipelines.tiles_pipes.EEZTilesPipe.EEZTilesPipe,
 'mpa_tiles': pipelines.tiles_pipes.MpasTilesPipe.MpasTilesPipe,
 'regions_tiles': pipelines.tiles_pipes.RegionsTilesPipe.RegionsTilesPipe,
 'eez_locations_precalc': pipelines.precalc_pipes.LocationsPipe.LocationsPipe,
 'mpa_coverage_precalc': pipelines.precalc_pipes.PACoveragePipe.PACoveragePipe}

In [4]:
mypipes_subset = filter_pipes(get_pipes(), ["mpaatlas_intermediate"])

In [5]:
for n, pipe in mypipes_subset.items():
    new_pipe = pipe()
    new_pipe.extract().transform() #.load()

DEBUG:pipelines.settings:/home/mambauser/data
INFO:pipelines.base_pipe:Pipeline mpaatlas_intermediate running at 2023-11-14 15:08:31.858058: starting extract...
INFO:utils:File /home/mambauser/data/mpaatlas_intermediate/mpatlas_assess_zone.geojson already exists.
INFO:pipelines.base_pipe:Pipeline mpaatlas_intermediate finish at 2023-11-14 15:08:31.859840: Success executing extract
INFO:pipelines.base_pipe:Pipeline mpaatlas_intermediate running at 2023-11-14 15:08:31.860352: starting transform...
/home/mambauser/src/pipelines/intermediate_pipes/MpaAtlasIntermediatePipe.py:120: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(
INFO:pipelines.base_pipe:Pipeline mpaatlas_intermediate finish at 2023-11-14 15:08:53.014880: Success executing transform
